In [ ]:
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def visualize_fn(fn, l=-10, r=10, n=1000):
    x = np.linspace(l, r, num=n)
    y = fn(x)
    plt.plot(x, y); plt.show()

In [ ]:
x = np.arange(10)
x[0] = 10
y = jnp.arange(10)

#Operação não permitida! Arrays são imutáveis
y[0] = 10

In [ ]:
z = jnp.arange(10)
w = z.at[0].set(100)
print(z)
print(w)

## Comparação de perfomance

In [ ]:

tamanho = 3000
x_jnp = jnp.arange(tamanho)
x_np = np.arange(tamanho)

%timeit np.dot(x_np, x_np.T)
%timeit jnp.dot(x_jnp, x_jnp.T).block_until_ready()

## Jit

In [ ]:
def relu(x):
  return jnp.array(x+10)

seed = 0
#Toda a geração de numeros aleatórios precisa de uma key
key = jax.random.PRNGKey(seed)

#Função aprimorada, tem mais desempenho
relu_jit = jax.jit(relu)

x = jax.random.normal(key, 1000)

%timeit relu_jit(x).block_until_ready()
%timeit relu(x).block_until_ready()




In [ ]:
# Jit não é valido para qualquer tipo de função!


## Grad

In [ ]:
# Agiliza a descida gradiente
# Modo automático

def mse(x):
  return jnp.sum((x**2)) # x1 * 2 + x2 * 2 + x3 * 2 + x4 * 2 -> Derivada da função UAU

x = jnp.arange(4.)
loss_func = mse


loss_func_grad = jax.grad(loss_func)

#Calcula automaticamente o gradiente, considerando que x é a diferença entre as saidas do modelo e as entradas esperadas
print(loss_func_grad(x))


In [ ]:
#Função com devirada indefinida para x = 0
func = lambda x: jnp.abs(x)

#Calcula o gradiente
func_grad = jax.grad(func)


visualize_fn(func)

#Define a derivada de 0 para 1, por convenção e praticidade
func_grad(0.)

## Vmap

In [ ]:
#Escrever suas próprias funções como se estivesse tratando com um escalar
W = jax.random.normal(key, (150, 150))
x_batched = jax.random.normal(key, (50, 150)) #Tratar como "batch", + RAPIDO

def mult_matrix(x):
  return jnp.dot(W, x)

In [ ]:
def vector_mult_matrix(x):
  return jnp.stack([mult_matrix(obj) for obj in x])

print("Vetorizado:")
%timeit vector_mult_matrix(x_batched).block_until_ready()

In [ ]:
#Usando jit
mult_matrix_jit = jax.jit(mult_matrix)

def vector_mult_matrix_jit(x):
  return jnp.stack([mult_matrix_jit(obj) for obj in x])

print("Vetorizado com jit:")
%timeit vector_mult_matrix_jit(x_batched)

In [ ]:
#Usando vmap

def vmap_mult_matrix(x):
  return jax.vmap(mult_matrix)(x)

#Perceba que a sintaxe é: jax.vmap(função aplicada a apenas um valor)(vetor)
%timeit vmap_mult_matrix(x_batched)

## Lax


> Numpy -> Lax -> XLA



In [ ]:
# Lax é mais restrititivo, porém mais rápido
print(jnp.add(1, 1.0))
print(jax.lax.add(1, 1.0))
#Erro! O tipo dos dados precisa ser os mesmos